## 규칙
- 각 이름은 띄어쓰기 없이 작성하기. 예를 들어, "영업 시간"이 아니라 "영업시간"으로

In [111]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
import pickle
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [112]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user09/beaver/data/db"
    data_path = ""
    save_path = ""
    embedding_model= "nlpai-lab/KoE5" # "BAAI/bge-m3"
    retriever_k=5
    retriever_bert_weight=0.7
    data_version='_menuoption_v1'
    version='_menuoption_v2_SQL'
    # info_type='STORE_INFO'   # STORE_INFO, TIME_INFO, MENU_INFO
    store_num = 105200  # 103017(미스), 104049(홍콩), 104562(새마을), 104570(롤링), 105200(우아)
    fill_nan = "None"  # 없는 정보의 경우에는 "정보없음"보다는 "None"이 나은 듯
    # basic_info = ["주차장", "씨씨티비", "영업시간", "예약가능여부", "전화번호"]
    seed=42 


CFG.data_path = f"/home/user09/beaver/data/shared_files/dataset/dataset{CFG.data_version}.xlsx"
print(CFG.data_path)
CFG.save_path = f"/home/user09/beaver/data/shared_files/dataset/dataset{CFG.version}_{CFG.store_num}_preprocessed.xlsx"
print(CFG.save_path)

/home/user09/beaver/data/shared_files/dataset/dataset_menuoption_v1.xlsx
/home/user09/beaver/data/shared_files/dataset/dataset_menuoption_v2_SQL_105200_preprocessed.xlsx


### STORE_INFO

In [113]:
################ 전처리 ################
from langchain.schema import Document

# ['STORE_NO', 'STORE_NM', 'BRAND_NO', 'BRAND_NM', 'ROAD_NM', 'BSPLC_TELNO', 'X', 'Y', 'T_CNT', 'P_CNT']
columns_to_use = ['STORE_NO', 'STORE_NM', 'ROAD_NM', 'BSPLC_TELNO', 'T_CNT', 'P_CNT']
df_store = pd.read_excel(CFG.data_path, sheet_name='STORE_INFO', usecols=columns_to_use, dtype={"BSPLC_TELNO": str})

df_store = df_store[df_store['STORE_NO']==CFG.store_num]  # 해당 매장 정보만 가져옴
store = df_store['STORE_NM'].unique()[0]
print(store)

df_store.drop(columns=['STORE_NO'], inplace=True)

df_store_col = df_store.columns

# 결과 출력
print(df_store)
print(df_store_col)

우아하계 완정역점
    STORE_NM           ROAD_NM  BSPLC_TELNO  T_CNT  P_CNT
4  우아하계 완정역점  인천 서구 완정로10번길 16  01099923537     18     72
Index(['STORE_NM', 'ROAD_NM', 'BSPLC_TELNO', 'T_CNT', 'P_CNT'], dtype='object')


In [114]:
############## 컬럼명 변경 ##############
# ['STORE_NO', 'STORE_NM', 'BRAND_NO', 'BRAND_NM', 'ROAD_NM', 'BSPLC_TELNO', 'X', 'Y', 'T_CNT', 'P_CNT']
store_column_mapping = {
    'STORE_NO': '상점번호',
    'STORE_NM': '매장명',
    'ROAD_NM': '위치',
    'BSPLC_TELNO': '전화번호',
    'X': '좌표 X',
    'Y': '좌표 Y',
    'T_CNT': '매장 내 테이블 개수',  # !!!!!!!!!!!!!!!!!
    'P_CNT': '수용 가능 인원',
}

for col in df_store_col:
    df_store = df_store.rename(columns={col: store_column_mapping[col]})
    
print(df_store)

         매장명                위치         전화번호  매장 내 테이블 개수  수용 가능 인원
4  우아하계 완정역점  인천 서구 완정로10번길 16  01099923537           18        72


In [115]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        for col in df.columns:
            contents_data.append(f'"{col}": {row[col]}')
    new_df = pd.DataFrame({'contents': contents_data})
    return new_df

# 함수 호출
result_df_store = create_contents_rowwise(df_store)
print(result_df_store)

                 contents
0        "매장명": 우아하계 완정역점
1  "위치": 인천 서구 완정로10번길 16
2     "전화번호": 01099923537
3       "매장 내 테이블 개수": 18
4          "수용 가능 인원": 72


### PATMNT_INFO

In [116]:
import pandas as pd

# 엑셀 파일에서 필요한 컬럼만 가져오기
columns_to_use = ['STORE_NO', 'PAYMNT_MN', 'EASY_PAYMNT_TYPE_NM']
df_pay = pd.read_excel(CFG.data_path, sheet_name='PAYMNT_INFO', usecols=columns_to_use)

# 매장 번호 필터링 (103017로 변경)
df_pay = df_pay[df_pay['STORE_NO'] == CFG.store_num]  # 매장 번호에 맞는 데이터만 가져옴
df_pay.drop(columns=['STORE_NO'], inplace=True)

# EASY_PAYMNT_TYPE_NM의 첫 번째 값 처리 (결측값 확인 후 진행)
# if not df_pay['EASY_PAYMNT_TYPE_NM'].dropna().empty:
simple_pay = str(df_pay['EASY_PAYMNT_TYPE_NM'].dropna().iloc[0])  # 첫 번째 비어있지 않은 값 가져오기


# PAYMNT_MN 컬럼을 문자열로 변환
df_pay['PAYMNT_MN'] = df_pay['PAYMNT_MN'].astype(str)

# 간편결제 문자열을 조건에 맞게 결합
# if simple_pay:  # simple_pay가 None이 아닐 때만 수행
#     df_pay.loc[df_pay['PAYMNT_MN'].str.strip() == '간편결제', 'PAYMNT_MN'] += "(" + simple_pay + ")"
# df_pay.drop(columns=['EASY_PAYMNT_TYPE_NM'], inplace=True)

df_pay_col = df_pay.columns

# 결과 출력
print(df_pay)
print(df_pay_col)

   PAYMNT_MN                                EASY_PAYMNT_TYPE_NM
28      신용카드                                                NaN
29        현금                                                NaN
30    모바일상품권                                                NaN
31       포인트                                                NaN
32      매장쿠폰                                                NaN
33       선불권                                                NaN
34      간편결제  네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드, 위챗페이, 카카오...
Index(['PAYMNT_MN', 'EASY_PAYMNT_TYPE_NM'], dtype='object')


In [117]:
############## 컬럼명 변경 ##############
# ['STORE_NO', 'PAYMNT_MN', 'EASY_PAYMNT_TYPE_NM']
pay_column_mapping = {
    'STORE_NO': '상점번호',
    'PAYMNT_MN': '결제수단',
    'EASY_PAYMNT_TYPE_NM': '간편결제수단',
}

for col in df_pay_col:
    df_pay = df_pay.rename(columns={col: pay_column_mapping[col]})

print(df_pay)

      결제수단                                             간편결제수단
28    신용카드                                                NaN
29      현금                                                NaN
30  모바일상품권                                                NaN
31     포인트                                                NaN
32    매장쿠폰                                                NaN
33     선불권                                                NaN
34    간편결제  네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드, 위챗페이, 카카오...


In [118]:
# 간편결제 행 찾기
easy_pay_row = df_pay[df_pay['결제수단'] == '간편결제']
if not easy_pay_row.empty:
    # 간편결제수단 값 추출
    easy_pay_method = easy_pay_row['간편결제수단'].iloc[0]
else:
    easy_pay_method = np.nan
    
payment_methods = df_pay['결제수단'].tolist()
payment_methods_str = ", ".join(payment_methods)

df_pay = pd.DataFrame({
    '결제수단': [payment_methods_str],
    '간편결제수단': [easy_pay_method if pd.notna(easy_pay_method) else '없음']
})

print(df_pay)

                                     결제수단  \
0  신용카드, 현금, 모바일상품권, 포인트, 매장쿠폰, 선불권, 간편결제   

                                              간편결제수단  
0  네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드, 위챗페이, 카카오...  


In [119]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        for col in df.columns:
            contents_data.append(f"'{col}': {row[col]}")
    new_df = pd.DataFrame({'contents': contents_data})
    return new_df

# 함수 호출
result_df_pay = create_contents_rowwise(df_pay)
print(result_df_pay)

                                            contents
0     '결제수단': 신용카드, 현금, 모바일상품권, 포인트, 매장쿠폰, 선불권, 간편결제
1  '간편결제수단': 네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드,...


### MENU_INFO

In [120]:
################ 전처리 ################
# ['STORE_NO', 'BRAND_NO', 'CATEGORY_NO', 'CATEGORY_NM', 'STD_CATEGORY_NM', 'PROD_NO', 'PROD_HIST_NO', 'PROD_NM', 'STD_PROD_NM', 'SET_PROD_YN', 'ORGIN_INDI', 'INDI_TYPE_NM', 'PROD_BASE_PRICE', 'QTY', 'OPTION_GROUP_NO', 'OPTION_GROUP_NM', 'MIN_SEL_CNT' ,'MAX_SEL_CNT', 'ESS_YN', 'OPTION_PROD_NO', 'OPTION_PROD_NM', 'OPTION_PROD_PRICE']
columns_to_use = ['STORE_NO', 'STD_CATEGORY_NM', 'INDI_TYPE_NM', 'STD_PROD_NM', 'PROD_BASE_PRICE', 'OPTION_PROD_NM', 'OPTION_PROD_PRICE']
df_menu = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset{CFG.data_version}.xlsx', sheet_name='MENU_INFO', usecols=columns_to_use)
df_menu = df_menu[df_menu['STORE_NO']==CFG.store_num]
df_menu.drop(columns=['STORE_NO'], inplace=True)
df_menu['INDI_TYPE_NM'].fillna("없음", inplace=True)

df_menu_col = df_menu.columns

print(df_menu[:3])
print(df_menu_col)

    STD_CATEGORY_NM STD_PROD_NM INDI_TYPE_NM  PROD_BASE_PRICE OPTION_PROD_NM  \
498            메인메뉴   우아하계 한판세트           없음            55000            NaN   
499            메인메뉴  숯불닭갈비갈비양념            없음            14000            NaN   
500            메인메뉴  숯불닭갈비소금구이            없음            14000            NaN   

     OPTION_PROD_PRICE  
498                NaN  
499                NaN  
500                NaN  
Index(['STD_CATEGORY_NM', 'STD_PROD_NM', 'INDI_TYPE_NM', 'PROD_BASE_PRICE',
       'OPTION_PROD_NM', 'OPTION_PROD_PRICE'],
      dtype='object')


In [121]:
menu_column_mapping = {
    'STORE_NO': '상점번호',
    'STD_PROD_NM': '메뉴명',
    'STD_CATEGORY_NM': '카테고리',
    'INDI_TYPE_NM': '특징',
    'PROD_BASE_PRICE': '가격',
    'OPTION_PROD_NM': '옵션',
    'OPTION_PROD_PRICE': '추가가격'
}

for col in df_menu_col:
    df_menu = df_menu.rename(columns={col: menu_column_mapping[col]})
    
print(df_menu[:5])

     카테고리         메뉴명  특징     가격   옵션  추가가격
498  메인메뉴   우아하계 한판세트  없음  55000  NaN   NaN
499  메인메뉴  숯불닭갈비갈비양념   없음  14000  NaN   NaN
500  메인메뉴  숯불닭갈비소금구이   없음  14000  NaN   NaN
501  메인메뉴      닭목살구이   없음  13000  NaN   NaN
502  메인메뉴    닭목살구이양념   없음  13000  NaN   NaN


In [122]:
# import pandas as pd
# import json

# # 결과를 담을 딕셔너리 (메뉴명 기준으로 그룹화)
# menu_group = {}

# for _, row in df_menu.iterrows():
#     # 특징 처리
#     if row['특징'] == "없음":
#         features = [row['카테고리']]
#     else:
#         # 특징과 카테고리를 콤마로 나눈 뒤 리스트로 합침
#         features = row['특징'].split(", ") + [row['카테고리']]

#     menu_name = row['메뉴명']
#     price = f"{row['가격']}원"

#     # 동일한 메뉴가 이미 그룹화 딕셔너리에 있는지 확인
#     if menu_name not in menu_group:
#         menu_group[menu_name] = {
#             "메뉴명": menu_name,
#             "가격": price,
#             "특징": features,
#             "옵션 및 추가금액": {}
#         }
#     else:
#         # 이미 있는 경우 특징을 추가적으로 합침 (중복 제거)
#         menu_group[menu_name]["특징"] = list(set(menu_group[menu_name]["특징"] + features))

#     # 추가가격 및 옵션 처리
#     if not pd.isna(row['추가가격']):
#         add_price = int(row['추가가격'])  # 정수로 변환
#         option_price = f"{add_price}원"
#         menu_group[menu_name]["옵션 및 추가금액"][row['옵션']] = option_price

# # 결과 리스트 생성
# result = []
# for menu in menu_group.values():
#     # 매칭되는 값에서 따옴표 제거
#     menu_str = (
#         '"메뉴명": ' + menu["메뉴명"] + ", "
#         '"가격": ' + menu["가격"] + ", "
#         '"특징": ' + str(menu["특징"]) + ", "
#         '"옵션 및 추가금액": ' + str(menu["옵션 및 추가금액"]).replace('"', "'")
#     )
#     result.append(menu_str)

# # 데이터프레임 생성
# result_df_menu = pd.DataFrame({"contents": result})

# # 출력
# print(result_df_menu[:5])
# print(result_df_menu['contents'][0])


In [123]:
import pandas as pd
import json

# 결과를 담을 딕셔너리 (메뉴명 기준으로 그룹화)
menu_group = {}

for _, row in df_menu.iterrows():
    # 특징 처리
    if row['특징'] == "없음":
        features = [row['카테고리']]
    else:
        # 특징과 카테고리를 콤마로 나눈 뒤 리스트로 합침
        features = row['특징'].split(", ") + [row['카테고리']]

    menu_name = row['메뉴명']
    price = f"{row['가격']}원"

    # 동일한 메뉴가 이미 그룹화 딕셔너리에 있는지 확인
    if menu_name not in menu_group:
        menu_group[menu_name] = {
            "메뉴명": menu_name,
            "가격": price,
            "특징": features,
        }
        # 옵션 및 추가금액 항목은 기본적으로 비활성화
        if not pd.isna(row['옵션']):
            menu_group[menu_name]["옵션 및 추가금액"] = {}

    else:
        # 이미 있는 경우 특징을 추가적으로 합침 (중복 제거)
        menu_group[menu_name]["특징"] = list(set(menu_group[menu_name]["특징"] + features))

    # 추가가격 및 옵션 처리
    if not pd.isna(row['추가가격']) and not pd.isna(row['옵션']):
        add_price = int(row['추가가격'])  # 정수로 변환
        option_price = f"{add_price}원"
        if "옵션 및 추가금액" not in menu_group[menu_name]:
            menu_group[menu_name]["옵션 및 추가금액"] = {}
        menu_group[menu_name]["옵션 및 추가금액"][row['옵션']] = option_price

# 결과 리스트 생성
result = []
for menu in menu_group.values():
    # 매칭되는 값에서 따옴표 제거
    menu_str = (
        '"메뉴명": ' + menu["메뉴명"] + ", "
        '"가격": ' + menu["가격"] + ", "
        '"특징": ' + str(menu["특징"])
    )
    # 옵션 및 추가금액이 존재하는 경우에만 추가
    if "옵션 및 추가금액" in menu:
        menu_str += ", " + '"옵션 및 추가금액": ' + str(menu["옵션 및 추가금액"]).replace('"', "'")
    result.append(menu_str)

# 데이터프레임 생성
result_df_menu = pd.DataFrame({"contents": result})

# 출력
print(result_df_menu[:5])
print(result_df_menu['contents'][20])


                                          contents
0   "메뉴명": 우아하계 한판세트, "가격": 55000원, "특징": ['메인메뉴']
1  "메뉴명": 숯불닭갈비갈비양념 , "가격": 14000원, "특징": ['메인메뉴']
2  "메뉴명": 숯불닭갈비소금구이 , "가격": 14000원, "특징": ['메인메뉴']
3      "메뉴명": 닭목살구이 , "가격": 13000원, "특징": ['메인메뉴']
4    "메뉴명": 닭목살구이양념 , "가격": 13000원, "특징": ['메인메뉴']
"메뉴명": 환타파인, "가격": 2000원, "특징": ['음료']


### TIME_INFO

In [124]:
# 전처리
columns_to_use = ['STORE_NO', 'WKDY', 'SALE_BGN_TM', 'SALE_END_TM', 'STORE_REST_BGN_TM', 'STORE_REST_END_TM']
df_time = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset{CFG.data_version}.xlsx', sheet_name='TIME_INFO', usecols=columns_to_use)
df_time = df_time[df_time['STORE_NO']==103017]

# 1. 요일 매핑
day_mapping = {
    2: '월요일',
    3: '화요일',
    4: '수요일',
    5: '목요일',
    6: '금요일',
    7: '토요일',
    1: '일요일',
    0: '매일'
}
df_time['WKDY'] = df_time['WKDY'].map(day_mapping)
df_time.fillna(CFG.fill_nan, inplace=True)

def convert_time_format(hhmmss):
    if pd.isna(hhmmss) or hhmmss in [None, 'None']:
        return "없음"
    hhmmss = str(int(hhmmss)).zfill(6)
    h = hhmmss[:2]
    m = hhmmss[2:4]
    if m == '00':
        return h + '시'
    else:
        return h + '시' + m + '분'

def handle_missing_values(df, columns):
    for col in columns:
        if df[col].isna().all():
            # 해당 매장의 해당 컬럼의 모든 값이 없으면, 정보가 없는 것으로 간주하여 None값으로 채움
            df[col] = CFG.fill_nan
        else:
            # 일부라도 값이 있으면 해당 요일에 브레이크 타임이 없는 경우 '없음'으로 채움
            df[col] = df[col].fillna("없음")
    return df

# 시간 변환 함수 적용
df_time['SALE_BGN_TM'] = df_time['SALE_BGN_TM'].apply(convert_time_format)
df_time['SALE_END_TM'] = df_time['SALE_END_TM'].apply(convert_time_format)
df_time['STORE_REST_BGN_TM'] = df_time['STORE_REST_BGN_TM'].apply(convert_time_format)
df_time['STORE_REST_END_TM'] = df_time['STORE_REST_END_TM'].apply(convert_time_format)

# 빈값 처리 함수 적용
columns_to_check = ['SALE_BGN_TM', 'SALE_END_TM', 'STORE_REST_BGN_TM', 'STORE_REST_END_TM']
df_time = handle_missing_values(df_time, columns_to_check)

# ===== 추가 로직 시작 =====
# 모든 요일(일~토:1~7)의 정보가 존재하며, 모든 시간이 동일한 경우 '매일(0)'로 통합
weekday_list = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일']
store_days = df_time['WKDY'].unique()

# 요일이 모두 7개 다 존재하는지 확인 (요일 순서는 중요치 않으므로 집합으로 비교)
if set(weekday_list).issubset(store_days):
    # 모든 요일에 대해 시간 정보가 동일한지 확인
    # 대표적으로 첫 번째 요일의 정보를 기준으로 비교
    df_first_day = df_time[df_time['WKDY'] == weekday_list[0]].iloc[0]
    same_time = True
    for w in weekday_list[1:]:
        df_current = df_time[df_time['WKDY'] == w].iloc[0]
        if (df_current['SALE_BGN_TM'] != df_first_day['SALE_BGN_TM'] or
            df_current['SALE_END_TM'] != df_first_day['SALE_END_TM'] or
            df_current['STORE_REST_BGN_TM'] != df_first_day['STORE_REST_BGN_TM'] or
            df_current['STORE_REST_END_TM'] != df_first_day['STORE_REST_END_TM']):
            same_time = False
            break
    
    # 동일하다면 매일(0)로 통합
    if same_time:
        # 매일(0)에 해당하는 행 추가
        daily_row = df_first_day.copy()
        daily_row['WKDY'] = '매일'
        
        # 기존 요일 행들 제거 후 매일 행만 남김
        df_time = df_time[~df_time['WKDY'].isin(weekday_list)]
        df_time = pd.concat([df_time, daily_row.to_frame().T], ignore_index=True)
df_time.drop(columns=['STORE_NO'], inplace=True)

print(df_time)


  WKDY SALE_BGN_TM SALE_END_TM STORE_REST_BGN_TM STORE_REST_END_TM
0   매일         07시      23시59분                없음                없음


In [125]:
# "영업시간" 컬럼 생성
df_time['영업시간'] = df_time['WKDY'] + " " + df_time['SALE_BGN_TM'] + "부터 " + df_time['SALE_END_TM'] + "까지"

# "브레이크타임" 컬럼 생성
def make_breaktime(row):
    start = row['STORE_REST_BGN_TM']
    end = row['STORE_REST_END_TM']
    if start == "없음" or start == "브레이크타임 없음" or end == "없음" or end == "브레이크타임 없음":
        return "없음"
    else:
        return start + "부터 " + end + "까지"

df_time['브레이크타임'] = df_time.apply(make_breaktime, axis=1)

# 필요없는 열들 정리(선택사항)
df_time = df_time[['영업시간', '브레이크타임']]

print(df_time)

                영업시간 브레이크타임
0  매일 07시부터 23시59분까지     없음


In [126]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        row_content = ", ".join([f'"{col}": {row[col]}' for col in df.columns])  # 각 행의 컬럼명:값을 ", "로 결합하여 하나의 문자열 생성
        contents_data.append(row_content)
    
    new_df = pd.DataFrame({'contents': contents_data})  # contents 열을 포함하는 새로운 데이터프레임 생성
    return new_df

# 함수 호출
result_df_time = create_contents_rowwise(df_time)
print(result_df_time[:3])

                                  contents
0  "영업시간": 매일 07시부터 23시59분까지, "브레이크타임": 없음


### 휴무일

In [127]:
# columns_to_use = ['STORE_NO', 'HOLIDAY_TYPE_CD']
# df_holiday = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx', sheet_name='HOLIDAY_INFO', usecols=columns_to_use)
# df_holiday = df_holiday[df_holiday['STORE_NO']==CFG.store_num]   # 해당 매장 정보만 가져옴
# df_holiday.fillna(CFG.fill_nan, inplace=True)
# df_holiday.drop(columns=['STORE_NO'], inplace=True)
# df_holiday_col = df_holiday.columns
# print(df_holiday)

In [128]:
# ############## 컬럼명 변경 ##############
# day_column_mapping = {
#     # 'PROD_NO': '상품 번호',
#     'HOLIDAY_TYPE_CD': '휴무일',
# }

# for col in df_holiday_col:
#     df_holiday = df_holiday.rename(columns={col: day_column_mapping[col]})
    
# print(df_holiday[:3])

In [129]:
# ############### 한 컬럼으로 통합 ###############
# def create_contents_rowwise(df):
#     contents_data = []
#     for _, row in df.iterrows():
#         row_content = ", ".join([f"'{col}': {row[col]}" for col in df.columns])  # 각 행의 컬럼명:값을 ", "로 결합하여 하나의 문자열 생성
#         contents_data.append(row_content)
    
#     new_df = pd.DataFrame({'contents': contents_data})  # contents 열을 포함하는 새로운 데이터프레임 생성
#     return new_df

# # 함수 호출
# result_df_holiday = create_contents_rowwise(df_holiday)
# print(result_df_holiday[:3])

### 4개 통합 & 피클파일

In [130]:
df = pd.concat([result_df_store, result_df_pay, result_df_menu, result_df_time], ignore_index=True)

In [131]:
print(df)

                                             contents
0                                    "매장명": 우아하계 완정역점
1                              "위치": 인천 서구 완정로10번길 16
2                                 "전화번호": 01099923537
3                                   "매장 내 테이블 개수": 18
4                                      "수용 가능 인원": 72
5      '결제수단': 신용카드, 현금, 모바일상품권, 포인트, 매장쿠폰, 선불권, 간편결제
6   '간편결제수단': 네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드,...
7      "메뉴명": 우아하계 한판세트, "가격": 55000원, "특징": ['메인메뉴']
8     "메뉴명": 숯불닭갈비갈비양념 , "가격": 14000원, "특징": ['메인메뉴']
9     "메뉴명": 숯불닭갈비소금구이 , "가격": 14000원, "특징": ['메인메뉴']
10        "메뉴명": 닭목살구이 , "가격": 13000원, "특징": ['메인메뉴']
11      "메뉴명": 닭목살구이양념 , "가격": 13000원, "특징": ['메인메뉴']
12       "메뉴명": 닭안창살구이 , "가격": 13000원, "특징": ['메인메뉴']
13     "메뉴명": 닭안창살구이양념 , "가격": 13000원, "특징": ['메인메뉴']
14      "메뉴명": 닭염통양념구이 , "가격": 13000원, "특징": ['메인메뉴']
15       "메뉴명": 무뼈닭발구이 , "가격": 13000원, "특징": ['메인메뉴']
16        "메뉴명": 한판세트 B, "가격": 39000원, "특징": ['메인메뉴']
17  "메뉴명": 능이닭개장, "가격": 9000

### 기본 정보 지정

In [132]:
# ":"를 기준으로 분리하여 앞의 키워드를 추출한 후, 존재 여부 확인
# existing_info = [content.split(":")[0].strip() for content in df["contents"]]
# missing_info = [info for info in CFG.basic_info if info not in existing_info]
# for info in missing_info:
#     df = pd.concat([df, pd.DataFrame({"contents": [f"{info}: 정보없음"]})], ignore_index=True)

In [133]:
# 결과 저장 및 출력
df.to_excel(CFG.save_path, index=False)   # 전처리한 엑셀 파일 저장
print(df)

                                             contents
0                                    "매장명": 우아하계 완정역점
1                              "위치": 인천 서구 완정로10번길 16
2                                 "전화번호": 01099923537
3                                   "매장 내 테이블 개수": 18
4                                      "수용 가능 인원": 72
5      '결제수단': 신용카드, 현금, 모바일상품권, 포인트, 매장쿠폰, 선불권, 간편결제
6   '간편결제수단': 네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드,...
7      "메뉴명": 우아하계 한판세트, "가격": 55000원, "특징": ['메인메뉴']
8     "메뉴명": 숯불닭갈비갈비양념 , "가격": 14000원, "특징": ['메인메뉴']
9     "메뉴명": 숯불닭갈비소금구이 , "가격": 14000원, "특징": ['메인메뉴']
10        "메뉴명": 닭목살구이 , "가격": 13000원, "특징": ['메인메뉴']
11      "메뉴명": 닭목살구이양념 , "가격": 13000원, "특징": ['메인메뉴']
12       "메뉴명": 닭안창살구이 , "가격": 13000원, "특징": ['메인메뉴']
13     "메뉴명": 닭안창살구이양념 , "가격": 13000원, "특징": ['메인메뉴']
14      "메뉴명": 닭염통양념구이 , "가격": 13000원, "특징": ['메인메뉴']
15       "메뉴명": 무뼈닭발구이 , "가격": 13000원, "특징": ['메인메뉴']
16        "메뉴명": 한판세트 B, "가격": 39000원, "특징": ['메인메뉴']
17  "메뉴명": 능이닭개장, "가격": 9000

In [134]:
#### 엑셀파일 DB화(pickle파일로 변환) ##### Extracting information from the "Details" column to create documents
docs = []
for _, row in df.iterrows():
    details = row['contents']
    # metadata = {}
    
    # # Parse key-value pairs in the "Details" column
    # for detail in details.split('\n'):
    #     if ':' in detail:
    #         key, value = detail.split(':', 1)
            # metadata[key.strip()] = value.strip()
    
    docs.append(Document(page_content=details))#, metadata=metadata))

# Embeddings and vector store setup
encode_kwargs = {'normalize_embeddings': True}
model_kwargs = {'device': 'cpu'}

hf = HuggingFaceEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

# Save the FAISS vector store and documents as pickle
db.save_local(f"{CFG.output_path}/{CFG.store_num}_faiss{CFG.version}")

with open(f"{CFG.output_path}/{CFG.store_num}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# Load the FAISS vector store
db = FAISS.load_local(f"/home/user09/beaver/data/db/{CFG.store_num}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)

# Create retrievers
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": CFG.retriever_k})
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight]
)
